In [1]:
!pip install holidayskr


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os
import numpy as np
import pandas as pd
import math
from tqdm import tqdm, tqdm_pandas # execution time
tqdm.pandas()

# Auto reload of library
%reload_ext autoreload
%autoreload 2

## DATA PROCESSING

In [3]:
import pandas as pd
import os
from tqdm import tqdm
from holidayskr import year_holidays

def preprocessing_KTX():
    df_seat, df_freq = pd.DataFrame(), pd.DataFrame()
    
    data_path = r'C:\Users\ckaud\Desktop\DEBAPROJECT\data'
    files = os.listdir(data_path)
    
    with tqdm(total=len(files), desc='Processing files') as pbar:
        for file in files:
            path_file = os.path.join(data_path, file)

            # 데이터 로딩
            if file[:10] == '(간선)시종착역별 ':
                df_sub = pd.read_excel(path_file, skiprows=8)
            else:
                df_sub = pd.read_excel(path_file, skiprows=5)

            # 데이터 결합
            if file[:10] == '(간선)수송-운행일':
                df_seat = pd.concat([df_seat, df_sub], axis=0)
            elif file[:10] == '(간선)시종착역별 ':
                df_freq = pd.concat([df_freq, df_sub], axis=0)
            
            pbar.update(1)
                
    # 불필요 변수 삭제
    columns_to_drop_seat = ['Unnamed: 1', '운행년도', '운행년월', '역무열차종', '메트릭']
    columns_to_drop_freq = ['상행하행구분', '역무열차종', '메트릭']
    
    df_seat.drop(columns=[col for col in columns_to_drop_seat if col in df_seat.columns], inplace=True)
    df_freq.drop(columns=[col for col in columns_to_drop_freq if col in df_freq.columns], inplace=True)
    
    # 주운행선 필터
    valid_lines = ['경부선', '경전선', '동해선', '호남선', '전라선']
    df_seat = df_seat[df_seat['주운행선'].isin(valid_lines)]
    df_freq = df_freq[df_freq['주운행선'].isin(valid_lines)]
    
    # index 재정렬
    df_seat = df_seat.reset_index(drop=True)
    df_freq = df_freq.reset_index(drop=True)
    
    # 시간변수 정의 및 주중여부 변수 생성
    df_seat['운행일자'] = pd.to_datetime(df_seat['운행일자'], format='%Y년 %m월 %d일')
    df_freq['운행일자'] = pd.to_datetime(df_freq['운행일자'], format='%Y년 %m월 %d일')
    df_seat['주중여부'] = df_seat['운행요일구분'].isin(['월', '화', '수', '목', '금']).astype(int)
    df_freq['주중여부'] = df_freq['운행요일구분'].isin(['월', '화', '수', '목', '금']).astype(int)
    df_seat.drop(columns=['운행요일구분'], inplace=True)
    df_freq.drop(columns=['운행요일구분'], inplace=True)
    
    # 공휴일 정보 미리 계산(년도 계산 후 일자별 비교)
    years = df_seat['운행일자'].dt.year.unique()
    holiday_dict = {}
    for year in years:
        holiday_dict[year] = {pd.to_datetime(day): name for day, name in year_holidays(year)}
    
    # 공휴일 여부 확인
    df_seat['공휴일여부'] = df_seat['운행일자'].apply(lambda x: int(x in holiday_dict[x.year]))
    df_freq['공휴일여부'] = df_freq['운행일자'].apply(lambda x: int(x in holiday_dict[x.year]))
    
    # 추가 변수 생성
    df_seat['승객별이동거리'] = df_seat['승차연인거리'] / df_seat['승차인원수']
    df_seat['좌석별이동거리'] = df_seat['좌석거리'] / df_seat['공급좌석합계수']
    df_seat['일별승차고객유형'] = df_seat['승차수입금액'] / df_seat['승차인원수']
    df_seat['일별승차판매율'] = df_seat['승차인원수'] / df_seat['공급좌석합계수']
    df_seat['일별승차비율'] = df_seat['승차연인거리'] / df_seat['좌석거리']
    df_seat['season'] = df_seat['운행일자'].dt.month % 12 // 3 + 1
    
    # 데이터 결합 및 추가 변수 생성해야 할 것
    # 월별실일자수 + 월별주중일수 + 월별휴일수 + 월별명절여부 
    # 일별/월별열차구분갯수 + 일별/월별열차속성갯수
    # 월별시발역nunique + 월별종착역nunique + 월별운행횟수 + 지연값(전월승차인원수)
    
    return df_seat, df_freq

df_seat, df_freq = preprocessing_KTX()

# 결과 확인
display(df_seat, df_freq)

Processing files: 100%|██████████████████████████████████████████████████████████████████| 4/4 [02:22<00:00, 35.56s/it]


,운행일자,주운행선,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,주중여부,공휴일여부,승객별이동거리,좌석별이동거리,일별승차고객유형,일별승차판매율,일별승차비율,season
0,2015-01-01,경부선,2016,104152,3725143600,101525,288685921,440665165,1,1,2843.495898,4230.981306,36691.884757,0.974777,0.655114,1
1,2015-01-01,경부선,38,2432,66411200,3563,8543090,10746496,1,1,2397.723828,4418.789474,18639.124333,1.465049,0.794965,1
2,2015-01-01,경부선,366,25812,518790400,65797,86492073,96079902,1,1,1314.529127,3722.295909,7884.712069,2.549086,0.900210,1
3,2015-01-01,경부선,40,2630,17088900,3988,2818725,3050800,1,1,706.801655,1160.000000,4285.080241,1.516350,0.923930,1
4,2015-01-01,경부선,112,5082,176309700,5142,13328413,19245897,1,1,2592.067872,3787.071429,34288.156359,1.011806,0.692533,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79536,2024-03-31,동해선,8,528,4456526,742,406147,895488,0,0,547.367925,1696.000000,6006.099730,1.405303,0.453548,2
79537,2024-03-31,경전선,216,11460,491723566,15533,36739684,48564615,0,0,2365.266465,4237.750000,31656.702891,1.355410,0.756511,2
79538,2024-03-31,경전선,10,436,4436700,679,820348,1418308,0,0,1208.170839,3253.000000,6534.167894,1.557339,0.578399,2
79539,2024-03-31,경전선,70,5008,31754080,7698,5292241,8450016,0,0,687.482593,1687.303514,4124.977916,1.537141,0.626300,2


,운행일자,열차속성,열차구분,주운행선,시발역,종착역,공급좌석수,설비좌석수,열차운행횟수,주중여부,공휴일여부
0,2015-01-01,일반,확정,경부선,서울,대전,931,931,1,1,1
1,2015-01-01,일반,확정,경부선,서울,대전,363,363,1,1,1
2,2015-01-01,일반,확정,경부선,서울,부산,35318,35318,38,1,1
3,2015-01-01,일반,확정,경부선,서울,부산,1089,1089,2,1,1
4,2015-01-01,일반,확정,경부선,대전,서울,931,931,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
281135,2024-03-31,일반/관광,확정,전라선,여수엑스포,용산,820,826,2,0,0
281136,2024-03-31,일반/관광,확정,전라선,여수엑스포,행신,1230,1239,3,0,0
281137,2024-03-31,일반/관광,확정,전라선,행신,여수엑스포,1640,1652,4,0,0
281138,2024-03-31,일반/관광,확정,동해선,서울,포항,820,826,2,0,0


## 월별집계

### df_seat 

In [32]:
def aggregate_monthly_by_line_and_date(df):
    # 월별 데이터로 묶기 위해 '운행일자'를 연-월 형식으로 변환
    df['연월'] = df['운행일자'].dt.to_period('M')
    
    # 계절 변수 생성
    df['season'] = df['운행일자'].dt.month % 12 // 3 + 1
    
    # 주운행선과 연월을 기준으로 그룹화하여 집계
    df_monthly = df.groupby(['주운행선', '연월']).agg({
        '승차수입금액': 'sum',
        '승차인원수': 'sum',
        '공급좌석합계수': 'sum',
        '승차연인거리': 'sum',
        '좌석거리': 'sum',
        '주중여부': 'mean',  # 주중여부는 평균으로 계산하여 주중인 날의 비율을 나타냄
        '공휴일여부': 'sum',  # 공휴일여부는 합계하여 월별 공휴일 수를 계산
        '승객별이동거리': 'mean',  # 승객별 이동거리의 평균
        '좌석별이동거리': 'mean',  # 좌석별 이동거리의 평균
        'season': 'first'  # 계절의 첫 번째 값을 사용
    }).reset_index()

    # 계절 더미 변수 생성 / 1: 겨울, 2: 봄, 3: 여름, 4: 가을
    df_monthly = pd.get_dummies(df_monthly, columns=['season'], prefix='season')

    # 추가적인 월별 통계 계산
    df_monthly['월별승차고객타입'] = df_monthly['승차수입금액'] / df_monthly['승차인원수']
    df_monthly['월별승차판매율'] = df_monthly['승차인원수'] / df_monthly['공급좌석합계수']
    df_monthly['월별승차비율'] = df_monthly['승차연인거리'] / df_monthly['좌석거리']
    
    return df_monthly

df_seat['승객별이동거리'] = df_seat['승차연인거리'] / df_seat['승차인원수']
df_seat['좌석별이동거리'] = df_seat['좌석거리'] / df_seat['공급좌석합계수']

df_monthly_seat = aggregate_monthly_by_line_and_date(df_seat)

# 결과 확인
display(df_monthly_seat)

,주운행선,연월,승차수입금액,승차인원수,공급좌석합계수,승차연인거리,좌석거리,주중여부,공휴일여부,승객별이동거리,좌석별이동거리,season_1,season_2,season_3,season_4,월별승차고객타입,월별승차판매율,월별승차비율
0,경부선,2015-01,148156008100,6462183,4690099,12887623030,18920136968,0.709677,6,1600.337218,3564.759324,1,0,0,0,22926.619085,1.377835,0.681159
1,경부선,2015-02,141350620300,6069139,4275701,12230649006,17328292972,0.714286,18,1607.155777,3587.031611,1,0,0,0,23290.061457,1.419449,0.705820
2,경부선,2015-03,139089651200,6565945,4643846,12032460501,18723639573,0.709677,6,1471.102989,3559.915460,0,1,0,0,21183.493191,1.413902,0.642635
3,경부선,2015-04,138289030400,6416662,4467804,11984070035,17902864800,0.761194,0,1658.216002,3356.580620,0,1,0,0,21551.552879,1.436200,0.669394
4,경부선,2015-05,155654821600,7082084,4687943,13565809400,18795445014,0.708738,20,1596.770583,3259.612114,0,1,0,0,21978.674865,1.510702,0.721760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,호남선,2023-11,37518557074,1658561,1628435,2892389782,4947939262,0.733333,0,1421.708803,3025.486343,0,0,0,1,22621.149945,1.018500,0.584565
551,호남선,2023-12,38790137140,1685725,1619297,2994256228,5122744713,0.677725,6,1480.917099,3208.834355,1,0,0,0,23010.952047,1.041023,0.584502
552,호남선,2024-01,35060217504,1493412,1528652,2703343618,5093271844,0.741935,6,1684.102405,3521.256853,1,0,0,0,23476.587508,0.976947,0.530768
553,호남선,2024-02,37464684994,1578740,1442514,2917783619,4815539060,0.719101,28,1724.013854,3542.169092,1,0,0,0,23730.750468,1.094437,0.605910


### df_freq

In [33]:
df_freq

,운행일자,열차속성,열차구분,주운행선,시발역,종착역,공급좌석수,설비좌석수,열차운행횟수,주중여부,공휴일여부
0,2015-01-01,일반,확정,경부선,서울,대전,931,931,1,1,1
1,2015-01-01,일반,확정,경부선,서울,대전,363,363,1,1,1
2,2015-01-01,일반,확정,경부선,서울,부산,35318,35318,38,1,1
3,2015-01-01,일반,확정,경부선,서울,부산,1089,1089,2,1,1
4,2015-01-01,일반,확정,경부선,대전,서울,931,931,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
281135,2024-03-31,일반/관광,확정,전라선,여수엑스포,용산,820,826,2,0,0
281136,2024-03-31,일반/관광,확정,전라선,여수엑스포,행신,1230,1239,3,0,0
281137,2024-03-31,일반/관광,확정,전라선,행신,여수엑스포,1640,1652,4,0,0
281138,2024-03-31,일반/관광,확정,동해선,서울,포항,820,826,2,0,0


In [50]:
def group_by_line_departure_and_month(df):
    # 운행일자를 연-월 형식으로 변환하여 '연월' 열 추가
    df['운행연월'] = df['운행일자'].dt.to_period('M')
    
    # 주어진 열로 그룹화
    df_grouped = df.groupby(['운행연월', '주운행선', '시발역', '종착역']).count()
    
    return df_grouped

df_seat, df_freq = preprocessing_KTX()
df_grouped_freq = group_by_line_departure_and_month(df_freq)

# 결과 확인
display(df_grouped_freq)

Processing files: 100%|██████████████████████████████████████████████████████████████████| 4/4 [03:20<00:00, 50.09s/it]


운행일자  열차속성  열차구분  공급좌석수  설비좌석수  열차운행횟수  주중여부  공휴일여부
운행연월    주운행선 시발역 종착역                                                        
2015-01 경부선  광명  부산        31    31    31     31     31      31    31     31
             대전  서울        62    62    62     62     62      62    62     62
                 인천공항T1    31    31    31     31     31      31    31     31
             부산  광명        31    31    31     31     31      31    31     31
                 서울        68    68    68     68     68      68    68     68
...                       ...   ...   ...    ...    ...     ...   ...    ...
2024-03 호남선  용산  익산        31    31    31     31     31      31    31     31
             익산  용산        31    31    31     31     31      31    31     31
                 행신        31    31    31     31     31      31    31     31
             행신  광주송정      31    31    31     31     31      31    31     31
                 목포       103   103   103    103    103     103   103    103

[4658 rows x 8 columns]

In [6]:
df_freq.groupby(['열차속성', '열차구분']).count()

운행일자    주운행선     시발역     종착역   공급좌석수   설비좌석수  열차운행횟수    주중여부
열차속성  열차구분                                                                
관광    임시         4       4       4       4       4       4       4       4
일반    대수송     7578    7578    7578    7578    7578    7578    7578    7578
      임시      4091    4091    4091    4091    4091    4091    4091    4091
      확정     81593   81593   81593   81593   81593   81593   81593   81593
일반/관광 대수송      278     278     278     278     278     278     278     278
      임시      2256    2256    2256    2256    2256    2256    2256    2256
      확정    185340  185340  185340  185340  185340  185340  185340  185340

In [7]:
# 주운행선별로 열차속성 및 열차구분으로 그룹화하여 count 열 생성
grouped = df_freq.groupby(['주운행선', '열차속성', '열차구분']).size().reset_index(name='count')

# 피벗 테이블 생성
pivot_df = pd.pivot_table(grouped, values='count', index='주운행선', columns=['열차속성', '열차구분'], aggfunc='sum', fill_value=0)

# 결과 출력
pivot_df

열차속성 관광    일반              일반/관광             
열차구분 임시   대수송    임시     확정   대수송    임시     확정
주운행선                                         
경부선   4  1567  1415  21434    80   682  38319
경전선   0  1470   346  16750    62   194  34486
동해선   0   291     0   1956     0     0   8603
전라선   0  1608   590  16061    50   241  37610
호남선   0  2642  1740  25392    86  1139  66322